# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [225]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [226]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [227]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [228]:
df=data.groupby(['CustomerID','ProductName'])[['Quantity']].sum().sort_values(ascending=False,by='Quantity')
df.head()

,,Quantity
CustomerID,ProductName,
90069,Longos - Grilled Salmon With Bbq,92
80694,Yeast Dry - Fermipan,84
96615,Veal - Eye Of Round,75
97063,Bread - French Baquette,75
95017,Coffee - Irish Cream,75


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [229]:
prodbycustomer = data.pivot_table(columns='CustomerID',values='Quantity', 
                                  index='ProductName', aggfunc='sum')

In [230]:
prodbycustomer.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Appetizer - Mini Egg Roll, Shrimp",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Appetizer - Mushroom Tart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN
Appetizer - Sausage Rolls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,25.0,25.0,25.0,NaN,25.0,NaN
Apricots - Dried,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
prodbycustomer = prodbycustomer.fillna(0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [232]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(prodbycustomer.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [233]:
distances = pd.DataFrame(1/(1 + squareform(pdist(prodbycustomer.T, 'euclidean'))), 
                        index=prodbycustomer.columns, columns=prodbycustomer.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [234]:
dist = distances[[33]].sort_values(ascending=False, by=33)

In [235]:
lista_index= list(dist[1:6].index)

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [236]:
records = df.loc[[264,3535,3317,2503,3305]]

In [237]:
records.head()

Quantity
CustomerID ProductName                            
3317       Pepper - Black, Whole                 2
3305       Wine - Ej Gallo Sierra Valley         2
           Sardines                              2
           Beans - Wax                           2
264        Guinea Fowl                           2

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [238]:
ranked = records.groupby(['ProductName'])[['Quantity']].sum().sort_values(ascending=False,by='Quantity')

In [241]:
ranked.head()

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [207]:
customer33 = df.loc[[33]]

In [243]:
list_products = pd.merge(ranked,customer33, how='left',on='ProductName')
list_products.head()

,Quantity_x,Quantity_y
ProductName,,
Butter - Unsalted,3,NaN
Wine - Ej Gallo Sierra Valley,3,NaN
Towels - Paper / Kraft,3,1.0
Soup - Campbells Bean Medley,3,NaN
Wine - Blue Nun Qualitatswein,3,NaN


In [244]:
list_products = list_products.fillna(0)

In [246]:
mask1= (list_products['Quantity_y']==0)
list_products[mask1].head()

,Quantity_x,Quantity_y
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [211]:
recommendations= {}
customer_ID_list = set(data['CustomerID'])

In [218]:
for customer in customer_ID_list:

    dist = distances[customer].sort_values(ascending=False) # step 4
    lista_index= list(dist[1:6].index)

    records = df.loc[lista_index] #step 5

    ranked = records.groupby(['ProductName']).sum().sort_values(ascending=False,by='Quantity') #step 6

    customer_number = df.loc[[customer]] #step 7
    list_products = pd.merge(ranked,customer_number, how='left',on='ProductName')
    list_products = list_products.fillna(0)
    mask= (list_products['Quantity_y']==0)
    recom = list_products[mask].head().index
    recommendations.update( {customer : list(recom)} )


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [186]:
pd.DataFrame.from_dict(recommendations, orient='index')

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
22536,Bacardi Breezer - Tropical,Cheese - Mozzarella,Cheese Cloth No 100,Bread - Italian Roll With Herbs,Bread - Hot Dog Buns
86028,Spinach - Baby,Pork - Hock And Feet Attached,Vinegar - Tarragon,Bread Foccacia Whole,Lettuce - Spring Mix
75791,Blueberries,Mussels - Cultivated,Vaccum Bag 10x13,Turkey - Oven Roast Breast,Bread - Calabrese Baguette
96272,Ecolab - Lime - A - Way 4/4 L,Bread Crumbs - Panko,Guinea Fowl,Sunflower Seed Raw,Bread - English Muffin
32785,Cookies - Assorted,Table Cloth 54x72 White,Juice - V8 Splash,Wine - Redchard Merritt,Puree - Passion Fruit


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [187]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(prodbycustomer.T, 'correlation'))), 
                        index=prodbycustomer.columns, columns=prodbycustomer.columns)

distances2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015


In [217]:
recommendations2={}
for customer in customer_ID_list:

        dist = distances2[customer].sort_values(ascending=False) # step 4
        lista_index= list(dist[1:6].index)

        records = df.loc[lista_index] #step 5

        ranked = records.groupby(['ProductName']).sum().sort_values(ascending=False,by='Quantity') #step 6

        customer_number = df.loc[[customer]] #step 7
        list_products = pd.merge(ranked,customer_number, how='left',on='ProductName')
        list_products = list_products.fillna(0)
        mask= (list_products['Quantity_y']==0)
        recom = list_products[mask].head().index
        recommendations2.update( {customer : list(recom)} )

In [219]:
recommendations[33]

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

In [220]:
recommendations2[33]

['Knife Plastic - White',
 'Muffin - Zero Transfat',
 'Banana Turning',
 'Crush - Cream Soda',
 'Veal - Osso Bucco']

In [221]:
recommendations[200]

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

In [222]:
recommendations2[200]

['Otomegusa Dashi Konbu',
 'Milk Powder',
 'Potatoes - Idaho 100 Count',
 'Crackers - Trio',
 'Pail With Metal Handle 16l White']